Связь "один ко многим" : одной записи главной таблицы соответствует несколько записей связанной таблицы, а каждой записи связанной таблицы соответствует только одна запись главной таблицы.

АВТОР -> КНИГА

Связь "многие ко многим" : каждой записи одной таблицы соответствует несколько записей во второй, и наоборот, каждой записи второй таблицы соответствует несколько записей в первой.

АВТОР <-> КНИГА

С помощью выражения ON DELETE можно установить действия, которые выполняются для записей подчиненной таблицы при удалении связанной строки из главной таблицы. При удалении можно установить следующие опции:

1. CASCADE: автоматически удаляет строки из зависимой таблицы при удалении  связанных строк в главной таблице.

2. SET NULL: при удалении  связанной строки из главной таблицы устанавливает для столбца внешнего ключа значение NULL. (В этом случае столбец внешнего ключа должен поддерживать установку NULL).

3. SET DEFAULT похоже на SET NULL за тем исключением, что значение  внешнего ключа устанавливается не в NULL, а в значение по умолчанию для данного столбца.

4. RESTRICT: отклоняет удаление строк в главной таблице при наличии связанных строк в зависимой таблице.

Пример: Будем считать, что при удалении автора из таблицы author, необходимо удалить все записи о книгах из таблицы book, написанные этим автором. Данное действие необходимо прописать при создании таблицы.

CREATE TABLE book (
    
    book_id INT PRIMARY KEY AUTO_INCREMENT, 
    
    title VARCHAR(50), 
    
    author_id INT NOT NULL, 
    
    price DECIMAL(8,2), 
    
    amount INT, 
    
    FOREIGN KEY (author_id)  REFERENCES author (author_id) ON DELETE CASCADE
);

# Запросы на выборку и соединение таблиц 
# JOIN

Оператор внутреннего соединения INNER JOIN соединяет две таблицы (порядок неважен)

SELECT
 ...
FROM
    таблица_1 INNER JOIN  таблица_2
    ON условие
...

Пример: Вывести название книг и их авторов

SELECT title, name_author
FROM 
    author INNER JOIN book
    
    ON author.author_id = book.author_id;


Оператор внешнего соединения LEFT OUTER JOIN  (можно использовать LEFT JOIN) соединяет две таблицы. Порядок таблиц для оператора важен.

SELECT
 ...
FROM
    таблица_1 LEFT JOIN  таблица_2
    ON условие
...

В LEFT JOIN добавляется доп инфа из первой таблицы, в RIGHT JOIN - из второй.


Оператор перекретного соединения, или декартова произведения CROSS JOIN соединяет две таблицы. Порядок таблиц неважен. Условия нет, поэтому результатом будут всевозможные комбинации

SELECT
 ...
FROM
    таблица_1 CROSS JOIN  таблица_2
... (и также вместо оператора можно поставить запятую)



Чтобы сгенерировать дату (в пределах 2020) - (DATE_ADD('2020-01-01', INTERVAL FLOOR(RAND() * 365) DAY)) as Дата

# Запросы для нескольких таблиц со вложенными запросами

Задача: Вывести авторов, общее количество книг которых на складе максимально.

SELECT name_author, SUM(amount) as Количество

FROM 
    
    author INNER JOIN book
    
    on author.author_id = book.author_id

GROUP BY name_author

HAVING SUM(amount) = 
     
     (/* вычисляем максимальное из общего количества книг каждого автора */
     
      SELECT MAX(sum_amount) AS max_sum_amount
     
      FROM 
     
          (/* считаем количество книг каждого автора */
     
            SELECT author_id, SUM(amount) AS sum_amount 
     
            FROM book GROUP BY author_id
     
          ) query_in
      );

Задание: Вывести в алфавитном порядке всех авторов, которые пишут только в одном жанре.

SELECT name_author

FROM

    author 

    INNER JOIN  book ON author.author_id = book.author_id

    INNER JOIN genre ON genre.genre_id = book.genre_id

GROUP BY name_author

HAVING COUNT(DISTINCT name_genre) = 1

ORDER BY name_author;

Задание: Вывести информацию о книгах (название книги, фамилию и инициалы автора, название жанра, цену и количество экземпляров книги), написанных в самых популярных жанрах, в отсортированном в алфавитном порядке по названию книг виде. Самым популярным считать жанр, общее количество экземпляров книг которого на складе максимально.

SELECT title, name_author, name_genre, price, amount

FROM author

INNER JOIN book ON author.author_id = book.author_id

INNER JOIN genre ON book.genre_id = genre.genre_id

GROUP BY title, name_author, name_genre, price, amount, genre.genre_id

HAVING genre.genre_id IN (SELECT queri_in_1.genre_id FROM

(SELECT genre_id, SUM(amount) AS summa FROM book

GROUP BY genre_id)queri_in_1

    INNER JOIN

(SELECT genre_id, SUM(amount) AS summa FROM book

GROUP BY genre_id

ORDER BY SUM(amount) DESC

LIMIT 1) queri_in_2

ON queri_in_1.summa = queri_in_2.summa

)

ORDER BY title;

*в группировку включен столбец genre_id, который используется в HAVING. Это связано с тем, что в HAVING можно использовать либо столбцы, перечисленные в GROUP BY, либо вычисляемые с помощью групповых функций столбцы. Добавление столбца genre_idне влияет на группировку, так как между названием жанра и его id - взаимно-однозначное соответствие.*

При описании соединения таблиц с помощью JOIN в некоторых случаях вместо ON и следующего за ним условия можно использовать оператор USING().

Пример:
*с использованием ON*

SELECT title, name_author, author.author_id /* явно указать таблицу - обязательно */

FROM 

    author INNER JOIN book

    ON author.author_id = book.author_id;

*c USING*

SELECT title, name_author, author_id /* имя таблицы, из которой берется author_id, указывать не обязательно*/

FROM 

    author INNER JOIN book

    USING(author_id);

# Запросы корректировки, соединение таблиц

ОБНОВЛЕНИЕ. Структура: 

UPDATE таблица_1
     ... JOIN таблица_2
     ON выражение
     ...
SET ...   
WHERE ...;

Пример: Для книг, которые уже есть на складе (в таблице book) по той же цене, что и в поставке (supply), увеличить количество на значение, указанное в поставке, а также обнулить количество этих книг в поставке.

UPDATE book 
     
     INNER JOIN author ON author.author_id = book.author_id
     
     INNER JOIN supply ON book.title = supply.title 
     
                         and supply.author = author.name_author

SET book.amount = book.amount + supply.amount,

    supply.amount = 0   

WHERE book.price = supply.price;

ВСТАВКА. Структура: 

INSERT INTO таблица (список_полей)
SELECT список_полей_из_других_таблиц
FROM 
    таблица_1 
    ... JOIN таблица_2 ON ...
    ...

Пример: Включить новых авторов в таблицу author с помощью запроса на добавление, а затем вывести все данные из таблицы author.  Новыми считаются авторы, которые есть в таблице supply, но нет в таблице author.

INSERT INTO author (name_author)

SELECT supply.author

FROM 

    author 

    RIGHT JOIN supply ON author.name_author = supply.author

WHERE name_author is null;

SELECT * FROM author

*в данном случае использовали right join, чтобы (новые) данные из таблицы supply, которых нет в author, были занесены в таблицу*